## Import Modules

In [1]:
import pdal
import sys
import os
import json
sys.path.append("../../../PythonScripts")
from glob import glob
import open3d as o3d
import os
import numpy as np
import pandas as pd
from multiprocessing import Pool
from pipeline_functions import downscale_las, get_scales,get_scales_cyl, downscale_ply, getFeatures,getFeaturesParallel,Cylinder
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from time import time
# from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree, BallTree
# from scipy.spatial import cKDTree
import pickle
from datetime import datetime, timezone
from itertools import repeat # Used for starmap function
import gc # Used to free up memory to prevent kernel restarting

sns.set()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
scls = get_scales_cyl(r_0=0.4, S = 6,gamma=1.5, rho = 8)

In [8]:
scls

[(0.5657, 0.0707),
 (0.8485, 0.1061),
 (1.2728, 0.1591),
 (1.9092, 0.2386),
 (2.8638, 0.358),
 (4.2957, 0.537)]

# Get number of processors to run loop

In [9]:
processors = os.cpu_count() - 4

## Get File to run

In [10]:
ROOT = """/home/sspiegel/CapstoneData/Ohio/dales_semantic_segmentation_las/dales_las/train/5080_54435.las"""

In [11]:
fiNm = ROOT.split("/")[-1].split(".")[0]

## Load in base point cloud

In [12]:
pipeline_json = [
    {
        "type": "readers.las",
        "filename": ROOT
    }
        
]


pipeline = pdal.Pipeline(json.dumps(pipeline_json))

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy = pipeline.arrays[0]

mta = pipeline.metadata



In [13]:
xyzT = np.array((xy["X"], xy["Y"], xy["Z"])).T

In [14]:
xyz = xyzT - np.min(xyzT, axis = 0)

## Get Classification

In [15]:
cls = xy["Classification"].astype(int)

In [16]:
scls

[(0.5657, 0.0707),
 (0.8485, 0.1061),
 (1.2728, 0.1591),
 (1.9092, 0.2386),
 (2.8638, 0.358),
 (4.2957, 0.537)]

## Instantiate point cloud

In [17]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [18]:
pc_dict = []
for s in scls:
    
    pc = pcd.voxel_down_sample(s[1])
    # pc_array = np.asarray(pc.points)
    pc_dict.append({
      "r" : s[0],
     "grid_size" : s[1],
     "point_cloud" : pc
     # "point_cloud_array" : pc_carray
    })


## Get scaling factors

### Downscale point clouds: Methodology

* Downscale point clouds based on multiscaling factor
* The scaling factor chosen from [Thomas Hugues et. al](https://ieeexplore.ieee.org/document/8490990/)
* Parameters
  * initial radius ($r_0$): 0.1
  * Number of scales ($S$): 6
  * Base exponent for expanding radius ($\gamma$): 2
  * scaling factor of grid ($\rho$): 5
  * radius of sphere for scale $s$: $r_s = r_0 * \gamma^{s}$
  * downsampled voxel size: $\frac{r_s}{\rho}$

* Use multiprocessing to speed up process

### Sample point cloud (first radius)

In [19]:
test = pc_dict[2]

test_pc = test["point_cloud"]

cyl = Cylinder(R = test['r'])

tree = KDTree(test_pc.points)
# tree = cKDTree(CylPoints)

In [20]:
print(test)

{'r': 1.2728, 'grid_size': 0.1591, 'point_cloud': PointCloud with 8841205 points.}


In [21]:
test_pc_arr = np.asarray(test_pc.points)

## Get Distance matrix

In [22]:
# cylPCDPoints = np.asarray(pcd.points)[:,:2]

In [23]:
distMat = tree.query_radius(pcd.points, r=test["r"])



# distMat = tree.query_ball_point(pcd.points, r=test["r"], workers = 1)


In [24]:
# distMat

In [25]:
# args1 = zip(repeat(pcOut),repeat(test_pc_arr), distMat)
# with Pool(processes=processors) as pool1:
#     result1 = pool1.starmap(cyl.computePoints, args1)
# distMatFix = np.array(result1)

In [26]:
pcOut = np.asarray(pcd.points)


In [27]:
del pc_dict, xyzT, xyz, xy, pcd

gc.collect()

1345

In [28]:
# cyl.computePointsParallel(pcOut[0],test_pc_arr,distMat[0])

In [29]:
args1 = zip(pcOut,repeat(test_pc_arr), distMat)
with Pool(processes=processors) as pool1:
    result1 = pool1.starmap(cyl.computePointsParallel, args1)
distMatFix = np.asarray(result1, dtype = "object")

In [30]:
gc.collect()

0

In [31]:
# distMat.dtype

In [32]:
# distMatFix = np.asarray(result1, dtype = "object")

In [33]:
# distMatFix[0]
# # 

In [34]:
# distMat[0]

In [35]:
# pcOut[0]

In [36]:
# ids = np.random.randint(0, pcOut.shape[0], 1)

In [37]:
# ids

In [38]:
# pcCenter = o3d.geometry.PointCloud()
# pcDown = o3d.geometry.PointCloud()
# pcCyl = o3d.geometry.PointCloud()

# pcCenter.points = o3d.utility.Vector3dVector(pcOut[ids[0]].reshape(1,-1))
# pcDown.points = o3d.utility.Vector3dVector(test_pc_arr[distMat[ids[0]]])
# pcCyl.points = o3d.utility.Vector3dVector(test_pc_arr[distMatFix[ids[0]]])

# pcCenter.paint_uniform_color([1,0,0])
# pcDown.paint_uniform_color([0,1,0])
# pcCyl.paint_uniform_color([0,0,1])

# o3d.visualization.draw_geometries([pcCenter, pcDown, pcCyl])


In [39]:
del distMat

In [40]:
gc.collect()

0

## Split and apply features algorithmKDTree

In [41]:
# DIV = 100000
# groups = distMatFix.shape[0] // DIV

In [42]:
s = time()

bigList = []

# for g in range(groups + 1):
#     print("""Processing batch %d...""" % (g+1))
#     practiceList = distMatFix[g*DIV:(g+1)*DIV]

args = zip(repeat(test_pc_arr), distMatFix)
with Pool(processes=processors) as pool:
    bigList = pool.starmap(getFeaturesParallel, args)
    # result = np.array(bigList)
    # bigList.append(result)
    # print(practiceList.shape)
    # break

bigList = np.array(bigList)
e = time()

print("""\n\n\n\nDone!!!  Total time: %.4f seconds""" % (e - s))





Done!!!  Total time: 292.4472 seconds


In [43]:
bigList[:3,:]

array([[3.50350547e-01, 2.49057020e-02, 6.05431242e-01, 3.59616580e-01,
        6.37895944e-01, 2.48747191e-03, 1.51410067e-03, 1.68982867e-02,
        1.55027110e+00, 5.10000000e+01],
       [2.84863931e-01, 2.14756644e-02, 5.43403327e-01, 4.91395297e-01,
        5.05695431e-01, 2.90926685e-03, 1.92473700e-03, 3.59610193e-02,
        1.53330972e+00, 3.90000000e+01],
       [3.36585207e-01, 2.22406394e-02, 5.93479399e-01, 3.78111393e-01,
        6.19902041e-01, 1.98656168e-03, 1.22334631e-03, 4.03665058e-02,
        1.52842961e+00, 5.50000000e+01]])

## Save as a pickle file

In [44]:
del distMatFix, tree

gc.collect()

0

In [45]:
# pickleOut = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}.pkl"""

In [46]:
getTime = datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")

featureOut = f"""/home/sspiegel/CapstoneData/Ohio/dales_semantic_segmentation_las/dales_las/train/pickleFiles/cylinder/{getTime}_{fiNm}_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.npz"""
# pickleOutPC = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{getTime}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_pointcloud.pkl"""

# outDict = { "features" : bigList, "classification" : cls}
# # outDict = {"PointCloud" : pcOut}


# with open(pickleOut, 'wb') as f:
#     pickle.dump(outDict, f)

np.savez(featureOut,array1 = pcOut, array2 = bigList, array3 = cls)

## Save stats

In [47]:
sts = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/stats/{datetime.now(timezone.utc).strftime("%Y_%m_%d")}_cylinder_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.json"""


In [48]:
jsStats = {
    "filename" : ROOT,
    "radius" : test["r"],
    "grid_size" : test["grid_size"],
    "total_time" : "%.4f seconds" % (e - s)
}

In [49]:
with open(sts, "w") as jsOut:
    jsOut.write(json.dumps(jsStats))
